<a href="https://colab.research.google.com/github/priyasha2011/quantum/blob/main/optimizedquantumVQC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# CELL 1: Install Dependencies (after restart)
!pip install pennylane torch torchvision pillow captcha numpy matplotlib -q
print("✅ Installation complete!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 144.2 MB/s eta 0:00:00
✅ Installation complete!


In [2]:
import os
import random
from captcha.image import ImageCaptcha
from tqdm import tqdm

CHAR_SET = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
MIN_LENGTH, MAX_LENGTH = 4, 6
IMG_WIDTH, IMG_HEIGHT = 180, 60
TRAIN_SIZE, VAL_SIZE, TEST_SIZE = 2000, 500, 500

BASE_DIR = "/content/captcha_dataset"
TRAIN_DIR = os.path.join(BASE_DIR, "train")
VAL_DIR = os.path.join(BASE_DIR, "val")
TEST_DIR = os.path.join(BASE_DIR, "test")

for path in [TRAIN_DIR, VAL_DIR, TEST_DIR]:
    os.makedirs(path, exist_ok=True)

def random_captcha_text(min_len=MIN_LENGTH, max_len=MAX_LENGTH):
    length = random.randint(min_len, max_len)
    return ''.join(random.choices(CHAR_SET, k=length))

def generate_dataset(split_dir, num_samples):
    image_captcha = ImageCaptcha(width=IMG_WIDTH, height=IMG_HEIGHT)
    for _ in tqdm(range(num_samples), desc=f"Generating {split_dir.split('/')[-1]}"):
        text = random_captcha_text()
        image = image_captcha.generate_image(text).convert("RGB")
        file_path = os.path.join(split_dir, f"{text}.png")
        image.save(file_path)

generate_dataset(TRAIN_DIR, TRAIN_SIZE)
generate_dataset(VAL_DIR, VAL_SIZE)
generate_dataset(TEST_DIR, TEST_SIZE)

print("✅ Dataset generation complete!")

Generating test: 100%|██████████| 500/500 [00:02<00:00, 171.38it/s]

✅ Dataset generation complete!


In [3]:
import os
from PIL import Image
from tqdm import tqdm

BASE_DIR = "/content/captcha_dataset"
SPLITS = ["train", "val", "test"]
OUT_SUBDIR = "chars_8x8"

for split in SPLITS:
    split_dir = os.path.join(BASE_DIR, split)
    out_dir = os.path.join(split_dir, OUT_SUBDIR)
    os.makedirs(out_dir, exist_ok=True)

    for fname in tqdm(sorted(os.listdir(split_dir)), desc=f"Segment {split}"):
        if not fname.lower().endswith(".png"):
            continue

        text = fname[:-4]  # drop .png
        img = Image.open(os.path.join(split_dir, fname)).convert("L")
        W, H = img.size
        n = len(text)
        slice_w = W / n

        for i, ch in enumerate(text):
            left = int(round(i * slice_w))
            right = int(round((i + 1) * slice_w))
            crop = img.crop((left, 0, right, H))
            crop = crop.resize((8, 8), Image.BILINEAR)
            save_name = f"{text}_{i}_{ch}.png"
            crop.save(os.path.join(out_dir, save_name))

print("✅ Character segmentation complete!")

Segment test: 100%|██████████| 501/501 [00:02<00:00, 239.47it/s]

✅ Character segmentation complete!


In [4]:
%%writefile dataset_chars_fixed.py

import os
from PIL import Image
import torch
from torch.utils.data import Dataset
import numpy as np

CHAR_SET = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

class CaptchaCharDataset(Dataset):
    """Load pre-segmented 8x8 character images from chars_8x8 subdirectory"""
    def __init__(self, root_dir, img_size=8):
        self.samples = []
        self.img_size = img_size

        # Look in the chars_8x8 subdirectory for segmented characters
        chars_dir = os.path.join(root_dir, "chars_8x8")
        if not os.path.exists(chars_dir):
            raise ValueError(f"Segmented chars directory not found: {chars_dir}")

        for fname in os.listdir(chars_dir):
            if fname.endswith('.png'):
                # Filename format: CAPTCHA_TEXT_position_character.png
                # Example: ABC123_0_A.png
                parts = fname[:-4].split('_')
                if len(parts) >= 3:
                    char_label = parts[-1]  # Last part is the character
                    if char_label in CHAR_SET:
                        self.samples.append((os.path.join(chars_dir, fname), char_label))

        self.char_to_idx = {ch: i for i, ch in enumerate(CHAR_SET)}
        print(f"Loaded {len(self.samples)} character samples from {chars_dir}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label_char = self.samples[idx]

        # Load the already-segmented 8x8 character image
        img = Image.open(img_path).convert('L')
        img = img.resize((self.img_size, self.img_size), Image.BILINEAR)

        # Normalize to [0, 1]
        x = np.array(img).flatten() / 255.0
        y = self.char_to_idx[label_char]

        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

Writing dataset_chars_fixed.py


In [5]:
# ===== CELL 5 (FIXED) =====
# Define Advanced Quantum Model with better error handling
%%writefile quantum_vqc_advanced.py

import pennylane as qml
import torch
import torch.nn as nn

CHAR_SET = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'

class FullyQuantumVQC(nn.Module):
    """
    Advanced Fully Quantum VQC with Data Re-uploading
    - 6 qubits, 3 layers
    - Multiple measurements (Z, ZZ, ZZZ)
    """
    def __init__(self, n_qubits=6, n_layers=3, n_classes=len(CHAR_SET)):
        super().__init__()
        self.n_qubits = n_qubits
        self.n_layers = n_layers
        self.n_classes = n_classes

        self.dev = qml.device("default.qubit", wires=n_qubits)

        @qml.qnode(self.dev, interface="torch", diff_method="backprop")
        def quantum_circuit(inputs, weights):
            # Ensure inputs are float32
            inputs = inputs.float()

            # Data re-uploading architecture
            for layer in range(n_layers):
                # Re-encode data at each layer
                qml.AmplitudeEmbedding(inputs, wires=range(n_qubits),
                                      pad_with=0.0, normalize=True)

                # Variational rotations
                for i in range(n_qubits):
                    qml.RY(weights[layer, i, 0], wires=i)
                    qml.RZ(weights[layer, i, 1], wires=i)

                # Entanglement
                for i in range(n_qubits - 1):
                    qml.CNOT(wires=[i, i + 1])
                qml.CNOT(wires=[n_qubits - 1, 0])

            # Multiple measurements
            measurements = []

            # Single-qubit Z measurements
            for i in range(n_qubits):
                measurements.append(qml.expval(qml.PauliZ(i)))

            # Two-qubit ZZ correlations
            for i in range(n_qubits - 1):
                measurements.append(qml.expval(qml.PauliZ(i) @ qml.PauliZ(i + 1)))

            # Three-qubit ZZZ correlations
            for i in range(n_qubits - 2):
                measurements.append(qml.expval(qml.PauliZ(i) @ qml.PauliZ(i + 1) @ qml.PauliZ(i + 2)))

            return measurements

        self.circuit = quantum_circuit
        self.q_weights = nn.Parameter(torch.randn(n_layers, n_qubits, 2, dtype=torch.float32) * 0.1)

        # Total measurements: 6 + 5 + 4 = 15
        n_measurements = n_qubits + (n_qubits - 1) + (n_qubits - 2)

        # Minimal classical output layer
        self.output_layer = nn.Linear(n_measurements, n_classes)

    def forward(self, x):
        batch_size = x.shape[0]
        quantum_features = []

        # Ensure input is float32
        x = x.float()

        for i in range(batch_size):
            try:
                measurements = self.circuit(x[i], self.q_weights)

                # Convert to tensor with explicit dtype
                if isinstance(measurements, (list, tuple)):
                    meas_list = []
                    for m in measurements:
                        if torch.is_tensor(m):
                            meas_list.append(m.detach().float())
                        else:
                            meas_list.append(torch.tensor(float(m), dtype=torch.float32))
                    measurements_tensor = torch.stack(meas_list).to(x.device)
                else:
                    measurements_tensor = measurements.detach().float()

                quantum_features.append(measurements_tensor)

            except Exception as e:
                print(f"Error in quantum circuit for sample {i}: {e}")
                # Return zeros as fallback
                n_measurements = self.n_qubits + (self.n_qubits - 1) + (self.n_qubits - 2)
                quantum_features.append(torch.zeros(n_measurements, dtype=torch.float32, device=x.device))

        quantum_features = torch.stack(quantum_features)
        logits = self.output_layer(quantum_features)

        return logits

Writing quantum_vqc_advanced.py


In [6]:
# ===== CELL 6 (RE-RUN) =====
# Reimport the fixed model
import sys
if 'quantum_vqc_advanced' in sys.modules:
    del sys.modules['quantum_vqc_advanced']

from quantum_vqc_advanced import FullyQuantumVQC
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Advanced model imported successfully!")
print(f"Using device: {device}")

# Test it thoroughly
test_model = FullyQuantumVQC(n_qubits=6, n_layers=3).to(device)
test_input = torch.randn(4, 64).to(device)
test_output = test_model(test_input)
print(f"Test passed! Output shape: {test_output.shape}")
print(f"Output dtype: {test_output.dtype}")
print(f"Output contains NaN: {torch.isnan(test_output).any()}")
print(f"Output contains Inf: {torch.isinf(test_output).any()}")

/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


✅ Advanced model imported successfully!
Using device: cuda
Test passed! Output shape: torch.Size([4, 36])
Output dtype: torch.float32
Output contains NaN: False
Output contains Inf: False


In [7]:
# ===== CELL 7 =====
# Prepare data loaders
from torch.utils.data import DataLoader
from dataset_chars_fixed import CaptchaCharDataset

train_dataset = CaptchaCharDataset("/content/captcha_dataset/train")
val_dataset = CaptchaCharDataset("/content/captcha_dataset/val")

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, num_workers=2)

print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

Loaded 9920 character samples from /content/captcha_dataset/train/chars_8x8
Loaded 2487 character samples from /content/captcha_dataset/val/chars_8x8
Training batches: 620
Validation batches: 156


In [8]:
# ===== CELL 8 (UPDATED WITH BETTER LOGGING) =====
# Training function with detailed epoch logging
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm

def train_quantum_model(model, train_loader, val_loader, device, epochs=50, lr=0.005, model_name="Advanced Quantum VQC"):
    model = model.to(device)
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    scheduler = CosineAnnealingLR(optimizer, T_max=epochs, eta_min=1e-5)

    best_val_acc = 0.0
    history = {'train_loss': [], 'train_acc': [], 'val_acc': [], 'lr': []}

    for epoch in range(epochs):
        # Training
        model.train()
        train_loss, train_correct, train_total = 0, 0, 0

        pbar = tqdm(train_loader, desc=f"[{model_name}] Epoch {epoch+1}/{epochs}")
        for batch_idx, (x, y) in enumerate(pbar):
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            logits = model(x)

            # Check for None or invalid output
            if logits is None:
                print(f"WARNING: Model returned None at batch {batch_idx}")
                continue

            loss = F.cross_entropy(logits, y)

            # Check for NaN loss
            if torch.isnan(loss):
                print(f"WARNING: NaN loss at batch {batch_idx}, skipping")
                continue

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()

            train_loss += loss.item()
            preds = logits.argmax(dim=1)
            train_correct += (preds == y).sum().item()
            train_total += y.size(0)

            # Update progress bar with current metrics
            current_acc = 100 * train_correct / train_total if train_total > 0 else 0
            current_lr = scheduler.get_last_lr()[0]
            pbar.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{current_acc:.2f}%',
                'lr': f'{current_lr:.6f}'
            })

        # Validation
        model.eval()
        val_correct, val_total = 0, 0
        val_loss = 0

        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                logits = model(x)

                if logits is not None:
                    loss = F.cross_entropy(logits, y)
                    val_loss += loss.item()

                    preds = logits.argmax(dim=1)
                    val_correct += (preds == y).sum().item()
                    val_total += y.size(0)

        train_acc = 100 * train_correct / train_total if train_total > 0 else 0
        val_acc = 100 * val_correct / val_total if val_total > 0 else 0
        avg_train_loss = train_loss / len(train_loader) if len(train_loader) > 0 else 0
        avg_val_loss = val_loss / len(val_loader) if len(val_loader) > 0 else 0
        current_lr = scheduler.get_last_lr()[0]

        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['lr'].append(current_lr)

        # Print detailed epoch summary (matching your desired format)
        print(f"\n[{model_name}] Epoch {epoch+1}/{epochs}:")
        print(f"  Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.2f}%")
        print(f"  LR: {current_lr:.6f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'val_acc': val_acc,
                'history': history
            }, 'best_advanced_quantum.pt')
            print(f"  ✅ New best model saved! Val Acc: {val_acc:.2f}%")

        scheduler.step()
        print("-" * 70)

    return history

print("✅ Training function with detailed logging ready!")

✅ Training function with detailed logging ready!


In [ ]:
# ===== CELL 9 (RE-RUN TRAINING) =====
# TRAIN THE ADVANCED MODEL
from quantum_vqc_advanced import FullyQuantumVQC

print("="*70)
print("TRAINING ADVANCED QUANTUM VQC MODEL")
print("  - 6 qubits with 64-amplitude encoding")
print("  - 3 layers with data re-uploading")
print("  - 15 quantum measurements (Z, ZZ, ZZZ)")
print("="*70)

model = FullyQuantumVQC(n_qubits=6, n_layers=3)

total_params = sum(p.numel() for p in model.parameters())
quantum_params = model.q_weights.numel()
print(f"\nTotal parameters: {total_params}")
print(f"Quantum parameters: {quantum_params}")
print(f"Classical parameters: {total_params - quantum_params}\n")

history = train_quantum_model(
    model,
    train_loader,
    val_loader,
    device,
    epochs=20,  # Change to 10 for quick test
    lr=0.005,
    model_name="Advanced Quantum VQC"
)

print(f"\n{'='*70}")
print(f"✅ Training complete!")
print(f"Best Validation Accuracy: {max(history['val_acc']):.2f}%")
print(f"{'='*70}")

TRAINING ADVANCED QUANTUM VQC MODEL
  - 6 qubits with 64-amplitude encoding
  - 3 layers with data re-uploading
  - 15 quantum measurements (Z, ZZ, ZZZ)

Total parameters: 612
Quantum parameters: 36
Classical parameters: 576



[Advanced Quantum VQC] Epoch 1/20: 100%|██████████| 620/620 [29:15<00:00,  2.83s/it, loss=3.5592, acc=4.59%, lr=0.005000]



[Advanced Quantum VQC] Epoch 1/20:
  Train Loss: 3.5672 | Train Acc: 4.59%
  Val Loss: 3.5502 | Val Acc: 5.59%
  LR: 0.005000
  ✅ New best model saved! Val Acc: 5.59%
----------------------------------------------------------------------


[Advanced Quantum VQC] Epoch 2/20: 100%|██████████| 620/620 [29:45<00:00,  2.88s/it, loss=3.5023, acc=6.85%, lr=0.004969]



[Advanced Quantum VQC] Epoch 2/20:
  Train Loss: 3.5281 | Train Acc: 6.85%
  Val Loss: 3.5190 | Val Acc: 6.35%
  LR: 0.004969
  ✅ New best model saved! Val Acc: 6.35%
----------------------------------------------------------------------


...

[Advanced Quantum VQC] Epoch 6/20: 100%|██████████| 620/620 [33:23<00:00,  3.23s/it, loss=3.4445, acc=8.48%, lr=0.004269]



[Advanced Quantum VQC] Epoch 6/20:
  Train Loss: 3.4415 | Train Acc: 8.48%
  Val Loss: 3.4517 | Val Acc: 8.08%
  LR: 0.004269
  ✅ New best model saved! Val Acc: 8.08%
----------------------------------------------------------------------


[Advanced Quantum VQC] Epoch 7/20: 100%|██████████| 620/620 [31:52<00:00,  3.08s/it, loss=3.4966, acc=8.85%, lr=0.003972]



[Advanced Quantum VQC] Epoch 7/20:
  Train Loss: 3.4293 | Train Acc: 8.85%
  Val Loss: 3.4427 | Val Acc: 7.88%
  LR: 0.003972
----------------------------------------------------------------------


[Advanced Quantum VQC] Epoch 8/20:  93%|█████████▎| 575/620 [29:47<02:15,  3.00s/it, loss=3.4816, acc=9.35%, lr=0.003638]

In [ ]:
# ===== CELL 10 =====
# Visualize results
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].plot(history['val_acc'], 'b-', linewidth=2.5, marker='o')
axes[0].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Validation Accuracy (%)', fontsize=12, fontweight='bold')
axes[0].set_title('Validation Accuracy', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=max(history['val_acc']), color='r', linestyle='--',
                label=f'Best: {max(history["val_acc"]):.2f}%')
axes[0].legend()

axes[1].plot(history['train_loss'], 'r-', linewidth=2.5, marker='s')
axes[1].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Training Loss', fontsize=12, fontweight='bold')
axes[1].set_title('Training Loss', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3)

axes[2].plot(history['train_acc'], 'g-', linewidth=2.5, marker='^')
axes[2].set_xlabel('Epoch', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Training Accuracy (%)', fontsize=12, fontweight='bold')
axes[2].set_title('Training Accuracy', fontsize=14, fontweight='bold')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('advanced_quantum_results.png', dpi=150)
plt.show()

print(f"\n{'='*70}")
print(f"FINAL RESULTS - ADVANCED QUANTUM VQC")
print(f"{'='*70}")
print(f"Best Validation Accuracy: {max(history['val_acc']):.2f}%")
print(f"Final Training Accuracy: {history['train_acc'][-1]:.2f}%")
print(f"Final Loss: {history['train_loss'][-1]:.4f}")
print(f"{'='*70}")